email agent
- authenticates user
    - only then are they allowed into the "inbox"
    - dynamic tools and prompt on the condition of there being an email and password in state that match hardcoded
- checks "inbox"
    - email in tool
- sends emails
    - human in the loop

In [3]:
from dotenv import load_dotenv

load_dotenv()


True

In [4]:
from langchain.chat_models import init_chat_model
model = init_chat_model(model="meta-llama/llama-4-maverick-17b-128e-instruct",
                        model_provider="groq",
                        temperature = 0,
                        max_retries=3,
                        timeout=60,
                        max_tokens=300
                        )

In [5]:
from dataclasses import dataclass

@dataclass
class EmailContext:
    email_address: str = "farhan@example.com"
    password: str = "password9"

In [6]:
from langchain.agents import AgentState

class AuthenticatedState(AgentState):
    authenticated: bool

In [7]:
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command
from langchain.messages import ToolMessage

@tool
def check_inbox() -> str:
    """Check the inbox for recent emails"""
    return """
    Hi Farhan, 
    I'm going to be in town next week and was wondering if we could grab a coffee?
    - best, farhan (farhan@example.com)
    """

@tool
def send_email(to: str, subject: str, body: str) -> str:
    """Send an response email"""
    return f"Email sent to {to} with subject {subject} and body {body}"

@tool
def authenticate(email: str, password: str, runtime: ToolRuntime) -> Command:
    """Authenticate the user with the given email and password"""
    if email == runtime.context.email_address and password == runtime.context.password:
        return Command(update={
            "authenticated": True, 
            "messages": [ToolMessage(
                "Successfully authenticated", 
                tool_call_id=runtime.tool_call_id)]
        })
    else:
        return Command(update={
            "authenticated": False,
            "messages": [ToolMessage(
                "Authentication failed", 
                tool_call_id=runtime.tool_call_id)]
        })

In [8]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable

@wrap_model_call
def dynamic_tool_call(request: ModelRequest, 
handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:

    """Allow read inbox and send email tools only if user provides correct email and password"""

    authenticated = request.state.get("authenticated")
    
    if authenticated:
        tools = [check_inbox, send_email]
    else:
        tools = [authenticate]

    request = request.override(tools=tools) 
    return handler(request)

In [9]:
from langchain.agents.middleware import dynamic_prompt

authenticated_prompt = "You are a helpful assistant that can check the inbox and send emails."
unauthenticated_prompt = "You are a helpful assistant that can authenticate users."

@dynamic_prompt
def dynamic_prompt(request: ModelRequest) -> str:
    """Generate system prompt based on authentication status"""
    authenticated = request.state.get("authenticated")

    if authenticated:
        return authenticated_prompt
    else:
        return unauthenticated_prompt

In [10]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware

agent = create_agent(
    model=model,
    tools=[authenticate, check_inbox, send_email],
    checkpointer=InMemorySaver(),
    state_schema=AuthenticatedState,
    context_schema=EmailContext,
    middleware=[
        dynamic_tool_call, 
        dynamic_prompt,
        HumanInTheLoopMiddleware(
            interrupt_on={
                "authenticate": False,
                "check_inbox": False,
                "send_email": True,
            })
        ]
    )


In [15]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [HumanMessage(content="farhan@example.com, password9")]},
    context=EmailContext(),
    config=config
)

print(response['messages'][-1].content)

c:\lca-lc-foundations\.venv\Lib\site-packages\pydantic\main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='context', input_value=EmailContext(email_addres...', password='password9'), input_type=EmailContext])
  return self.__pydantic_serializer__.to_python(


You have an email from farhan@example.com regarding meeting up for coffee.


In [17]:
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

KeyError: '__interrupt__'

In [18]:
from langgraph.types import Command

response = agent.invoke(
    Command( 
        resume={"decisions": [{"type": "approve"}]}  # or "reject"
    ), 
    config=config # Same thread ID to resume the paused conversation
)

print(response["messages"][-1].content)

You have an email from farhan@example.com regarding meeting up for coffee.


In [19]:
from pprint import pprint

pprint(response)

{'authenticated': True,
 'messages': [HumanMessage(content='draft 1', additional_kwargs={}, response_metadata={}, id='5439dab3-7a7e-4018-b655-38685adf24e3'),
              AIMessage(content='I don\'t have enough information to authenticate you. Please provide your email and password. \n\nTo proceed, I need the following information: \n- email \n- password \n\nYou can provide this information, and I\'ll be happy to assist you with authentication using the available "authenticate" function. \n\nTo authenticate, the input should be in the format:', additional_kwargs={'tool_calls': [{'id': 'c74hemakd', 'function': {'arguments': '{"email":"your_email","password":"your_password"}', 'name': 'authenticate'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 105, 'prompt_tokens': 728, 'total_tokens': 833, 'completion_time': 0.165375027, 'completion_tokens_details': None, 'prompt_time': 0.015559561, 'prompt_tokens_details': None, 'queue_time': 0.049245789, 'total_tim